# Comprehensive Resource Cleanup

This notebook provides a comprehensive cleanup procedure for all AWS resources created by the MLOps SageMaker Demo. It follows the AWS reference implementation's best practices for resource cleanup to prevent ongoing costs.

## Importance of Cleanup

AWS resources continue to incur costs until they are explicitly deleted. This is especially important for:

- SageMaker endpoints (which run continuously)
- SageMaker notebook instances (which run until stopped)
- S3 buckets with data storage
- CloudWatch logs and metrics

## Cleanup Process

This notebook will guide you through the following cleanup steps:

1. Delete SageMaker endpoints
2. Delete SageMaker models
3. Delete SageMaker pipelines
4. Delete monitoring schedules
5. Delete CloudWatch dashboards and alarms
6. Delete EventBridge rules
7. Empty S3 buckets
8. Delete CloudFormation stacks

Let's start by importing the necessary libraries and setting up our environment.

In [ ]:
import os
import boto3
import pandas as pd
import time
from datetime import datetime
from IPython.display import display, HTML
import ipywidgets as widgets
import json
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger('cleanup')

# Load project configuration
import sys
sys.path.append('..')
from configs.project_config import PROJECT_NAME, DATA_BUCKET

# Create AWS session with "ab" profile
session = boto3.Session(profile_name='ab')
account_id = session.client('sts').get_caller_identity()['Account']
region = session.region_name

print(f"Project: {PROJECT_NAME}")
print(f"AWS Account ID: {account_id}")
print(f"Region: {region}")

## Resource Discovery

Let's first discover all the resources created by the MLOps SageMaker Demo project. We'll use resource tagging to identify resources that belong to the project.

In [ ]:
# Create function to discover resources
def discover_resources():
    resources = {
        'endpoints': [],
        'models': [],
        'pipelines': [],
        'monitoring_schedules': [],
        'dashboards': [],
        'alarms': [],
        'rules': [],
        'stacks': []
    }
    
    # Create clients
    sagemaker_client = session.client('sagemaker')
    cloudwatch_client = session.client('cloudwatch')
    events_client = session.client('events')
    cf_client = session.client('cloudformation')
    
    # Discover SageMaker endpoints
    print("Discovering SageMaker endpoints...")
    try:
        response = sagemaker_client.list_endpoints()
        for endpoint in response['Endpoints']:
            if PROJECT_NAME.lower() in endpoint['EndpointName'].lower():
                resources['endpoints'].append({
                    'name': endpoint['EndpointName'],
                    'arn': endpoint['EndpointArn'],
                    'status': endpoint['EndpointStatus'],
                    'created': endpoint['CreationTime'].strftime('%Y-%m-%d %H:%M:%S')
                })
    except Exception as e:
        logger.error(f"Error discovering endpoints: {e}")
    
    # Discover SageMaker models
    print("Discovering SageMaker models...")
    try:
        response = sagemaker_client.list_models()
        for model in response['Models']:
            if PROJECT_NAME.lower() in model['ModelName'].lower():
                resources['models'].append({
                    'name': model['ModelName'],
                    'arn': model['ModelArn'],
                    'created': model['CreationTime'].strftime('%Y-%m-%d %H:%M:%S')
                })
    except Exception as e:
        logger.error(f"Error discovering models: {e}")
    
    # Discover SageMaker pipelines
    print("Discovering SageMaker pipelines...")
    try:
        response = sagemaker_client.list_pipelines()
        for pipeline in response['PipelineSummaries']:
            if PROJECT_NAME.lower() in pipeline['PipelineName'].lower():
                resources['pipelines'].append({
                    'name': pipeline['PipelineName'],
                    'arn': pipeline['PipelineArn'],
                    'created': pipeline['CreationTime'].strftime('%Y-%m-%d %H:%M:%S')
                })
    except Exception as e:
        logger.error(f"Error discovering pipelines: {e}")
    
    # Discover monitoring schedules
    print("Discovering monitoring schedules...")
    try:
        response = sagemaker_client.list_monitoring_schedules()
        for schedule in response['MonitoringScheduleSummaries']:
            if PROJECT_NAME.lower() in schedule['MonitoringScheduleName'].lower():
                resources['monitoring_schedules'].append({
                    'name': schedule['MonitoringScheduleName'],
                    'status': schedule['MonitoringScheduleStatus'],
                    'created': schedule['CreationTime'].strftime('%Y-%m-%d %H:%M:%S')
                })
    except Exception as e:
        logger.error(f"Error discovering monitoring schedules: {e}")
    
    # Discover CloudWatch dashboards
    print("Discovering CloudWatch dashboards...")
    try:
        response = cloudwatch_client.list_dashboards()
        for dashboard in response['DashboardEntries']:
            if PROJECT_NAME.lower() in dashboard['DashboardName'].lower():
                resources['dashboards'].append({
                    'name': dashboard['DashboardName'],
                    'arn': dashboard['DashboardArn'],
                    'last_modified': dashboard['LastModified'].strftime('%Y-%m-%d %H:%M:%S')
                })
    except Exception as e:
        logger.error(f"Error discovering dashboards: {e}")
    
    # Discover CloudWatch alarms
    print("Discovering CloudWatch alarms...")
    try:
        response = cloudwatch_client.describe_alarms()
        for alarm in response['MetricAlarms']:
            if PROJECT_NAME.lower() in alarm['AlarmName'].lower():
                resources['alarms'].append({
                    'name': alarm['AlarmName'],
                    'arn': alarm['AlarmArn'],
                    'state': alarm['StateValue']
                })
    except Exception as e:
        logger.error(f"Error discovering alarms: {e}")
    
    # Discover EventBridge rules
    print("Discovering EventBridge rules...")
    try:
        response = events_client.list_rules()
        for rule in response['Rules']:
            if PROJECT_NAME.lower() in rule['Name'].lower():
                resources['rules'].append({
                    'name': rule['Name'],
                    'arn': rule['Arn'],
                    'state': rule['State']
                })
    except Exception as e:
        logger.error(f"Error discovering rules: {e}")
    
    # Discover CloudFormation stacks
    print("Discovering CloudFormation stacks...")
    try:
        response = cf_client.list_stacks(StackStatusFilter=['CREATE_COMPLETE', 'UPDATE_COMPLETE', 'ROLLBACK_COMPLETE'])
        for stack in response['StackSummaries']:
            if PROJECT_NAME.lower() in stack['StackName'].lower():
                resources['stacks'].append({
                    'name': stack['StackName'],
                    'id': stack['StackId'],
                    'status': stack['StackStatus'],
                    'created': stack['CreationTime'].strftime('%Y-%m-%d %H:%M:%S')
                })
    except Exception as e:
        logger.error(f"Error discovering stacks: {e}")
    
    return resources

# Create button to discover resources
discover_button = widgets.Button(
    description='Discover Resources',
    button_style='info',
    tooltip='Click to discover resources'
)

discover_output = widgets.Output()

def on_discover_clicked(b):
    with discover_output:
        discover_output.clear_output()
        resources = discover_resources()
        
        # Display resources
        for resource_type, items in resources.items():
            if items:
                print(f"\n{resource_type.replace('_', ' ').title()} ({len(items)}):\n")
                df = pd.DataFrame(items)
                display(df)
            else:
                print(f"\n{resource_type.replace('_', ' ').title()}: None found")

discover_button.on_click(on_discover_clicked)

display(discover_button, discover_output)

## Resource Cleanup

Now, let's create functions to clean up each type of resource. We'll provide options to clean up resources selectively or all at once.

In [ ]:
# Create cleanup functions for each resource type

def delete_sagemaker_endpoints():
    """Delete SageMaker endpoints."""
    sagemaker_client = session.client('sagemaker')
    
    # Discover endpoints
    response = sagemaker_client.list_endpoints()
    endpoints = [endpoint for endpoint in response['Endpoints'] 
                if PROJECT_NAME.lower() in endpoint['EndpointName'].lower()]
    
    if not endpoints:
        print("No SageMaker endpoints found")
        return
    
    print(f"Found {len(endpoints)} SageMaker endpoints to delete")
    
    # Delete endpoints
    for endpoint in endpoints:
        endpoint_name = endpoint['EndpointName']
        try:
            print(f"Deleting endpoint: {endpoint_name}")
            sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
            print(f"Successfully deleted endpoint: {endpoint_name}")
        except Exception as e:
            print(f"Error deleting endpoint {endpoint_name}: {e}")

def delete_sagemaker_endpoint_configs():
    """Delete SageMaker endpoint configurations."""
    sagemaker_client = session.client('sagemaker')
    
    # Discover endpoint configs
    response = sagemaker_client.list_endpoint_configs()
    endpoint_configs = [config for config in response['EndpointConfigs'] 
                       if PROJECT_NAME.lower() in config['EndpointConfigName'].lower()]
    
    if not endpoint_configs:
        print("No SageMaker endpoint configurations found")
        return
    
    print(f"Found {len(endpoint_configs)} SageMaker endpoint configurations to delete")
    
    # Delete endpoint configs
    for config in endpoint_configs:
        config_name = config['EndpointConfigName']
        try:
            print(f"Deleting endpoint config: {config_name}")
            sagemaker_client.delete_endpoint_config(EndpointConfigName=config_name)
            print(f"Successfully deleted endpoint config: {config_name}")
        except Exception as e:
            print(f"Error deleting endpoint config {config_name}: {e}")

def delete_sagemaker_models():
    """Delete SageMaker models."""
    sagemaker_client = session.client('sagemaker')
    
    # Discover models
    response = sagemaker_client.list_models()
    models = [model for model in response['Models'] 
             if PROJECT_NAME.lower() in model['ModelName'].lower()]
    
    if not models:
        print("No SageMaker models found")
        return
    
    print(f"Found {len(models)} SageMaker models to delete")
    
    # Delete models
    for model in models:
        model_name = model['ModelName']
        try:
            print(f"Deleting model: {model_name}")
            sagemaker_client.delete_model(ModelName=model_name)
            print(f"Successfully deleted model: {model_name}")
        except Exception as e:
            print(f"Error deleting model {model_name}: {e}")

def delete_sagemaker_pipelines():
    """Delete SageMaker pipelines."""
    sagemaker_client = session.client('sagemaker')
    
    # Discover pipelines
    response = sagemaker_client.list_pipelines()
    pipelines = [pipeline for pipeline in response['PipelineSummaries'] 
                if PROJECT_NAME.lower() in pipeline['PipelineName'].lower()]
    
    if not pipelines:
        print("No SageMaker pipelines found")
        return
    
    print(f"Found {len(pipelines)} SageMaker pipelines to delete")
    
    # Delete pipelines
    for pipeline in pipelines:
        pipeline_name = pipeline['PipelineName']
        try:
            print(f"Deleting pipeline: {pipeline_name}")
            sagemaker_client.delete_pipeline(PipelineName=pipeline_name)
            print(f"Successfully deleted pipeline: {pipeline_name}")
        except Exception as e:
            print(f"Error deleting pipeline {pipeline_name}: {e}")

def delete_monitoring_schedules():
    """Delete SageMaker monitoring schedules."""
    sagemaker_client = session.client('sagemaker')
    
    # Discover monitoring schedules
    response = sagemaker_client.list_monitoring_schedules()
    schedules = [schedule for schedule in response['MonitoringScheduleSummaries'] 
                if PROJECT_NAME.lower() in schedule['MonitoringScheduleName'].lower()]
    
    if not schedules:
        print("No monitoring schedules found")
        return
    
    print(f"Found {len(schedules)} monitoring schedules to delete")
    
    # Delete monitoring schedules
    for schedule in schedules:
        schedule_name = schedule['MonitoringScheduleName']
        try:
            print(f"Deleting monitoring schedule: {schedule_name}")
            sagemaker_client.delete_monitoring_schedule(MonitoringScheduleName=schedule_name)
            print(f"Successfully deleted monitoring schedule: {schedule_name}")
        except Exception as e:
            print(f"Error deleting monitoring schedule {schedule_name}: {e}")

def delete_cloudwatch_dashboards():
    """Delete CloudWatch dashboards."""
    cloudwatch_client = session.client('cloudwatch')
    
    # Discover dashboards
    response = cloudwatch_client.list_dashboards()
    dashboards = [dashboard for dashboard in response['DashboardEntries'] 
                 if PROJECT_NAME.lower() in dashboard['DashboardName'].lower()]
    
    if not dashboards:
        print("No CloudWatch dashboards found")
        return
    
    print(f"Found {len(dashboards)} CloudWatch dashboards to delete")
    
    # Delete dashboards
    for dashboard in dashboards:
        dashboard_name = dashboard['DashboardName']
        try:
            print(f"Deleting dashboard: {dashboard_name}")
            cloudwatch_client.delete_dashboards(DashboardNames=[dashboard_name])
            print(f"Successfully deleted dashboard: {dashboard_name}")
        except Exception as e:
            print(f"Error deleting dashboard {dashboard_name}: {e}")

def delete_cloudwatch_alarms():
    """Delete CloudWatch alarms."""
    cloudwatch_client = session.client('cloudwatch')
    
    # Discover alarms
    response = cloudwatch_client.describe_alarms()
    alarms = [alarm for alarm in response['MetricAlarms'] 
             if PROJECT_NAME.lower() in alarm['AlarmName'].lower()]
    
    if not alarms:
        print("No CloudWatch alarms found")
        return
    
    print(f"Found {len(alarms)} CloudWatch alarms to delete")
    
    # Delete alarms
    alarm_names = [alarm['AlarmName'] for alarm in alarms]
    try:
        print(f"Deleting {len(alarm_names)} alarms")
        cloudwatch_client.delete_alarms(AlarmNames=alarm_names)
        print(f"Successfully deleted {len(alarm_names)} alarms")
    except Exception as e:
        print(f"Error deleting alarms: {e}")

def delete_eventbridge_rules():
    """Delete EventBridge rules."""
    events_client = session.client('events')
    
    # Discover rules
    response = events_client.list_rules()
    rules = [rule for rule in response['Rules'] 
            if PROJECT_NAME.lower() in rule['Name'].lower()]
    
    if not rules:
        print("No EventBridge rules found")
        return
    
    print(f"Found {len(rules)} EventBridge rules to delete")
    
    # Delete rules
    for rule in rules:
        rule_name = rule['Name']
        try:
            # Remove targets first
            targets_response = events_client.list_targets_by_rule(Rule=rule_name)
            if targets_response['Targets']:
                target_ids = [target['Id'] for target in targets_response['Targets']]
                print(f"Removing {len(target_ids)} targets from rule: {rule_name}")
                events_client.remove_targets(Rule=rule_name, Ids=target_ids)
            
            # Delete rule
            print(f"Deleting rule: {rule_name}")
            events_client.delete_rule(Name=rule_name)
            print(f"Successfully deleted rule: {rule_name}")
        except Exception as e:
            print(f"Error deleting rule {rule_name}: {e}")

def empty_s3_bucket(bucket_name):
    """Empty S3 bucket."""
    s3_client = session.client('s3')
    
    try:
        # Check if bucket exists
        s3_client.head_bucket(Bucket=bucket_name)
    except Exception as e:
        print(f"Bucket {bucket_name} does not exist or you don't have access: {e}")
        return
    
    print(f"Emptying bucket: {bucket_name}")
    
    # List and delete objects
    paginator = s3_client.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket_name)
    
    delete_list = []
    for page in pages:
        if 'Contents' in page:
            for obj in page['Contents']:
                delete_list.append({'Key': obj['Key']})
    
    if not delete_list:
        print(f"Bucket {bucket_name} is already empty")
        return
    
    # Delete objects in batches of 1000
    for i in range(0, len(delete_list), 1000):
        batch = delete_list[i:i+1000]
        s3_client.delete_objects(
            Bucket=bucket_name,
            Delete={'Objects': batch}
        )
        print(f"Deleted {len(batch)} objects from bucket")
    
    # List and delete object versions
    paginator = s3_client.get_paginator('list_object_versions')
    pages = paginator.paginate(Bucket=bucket_name)
    
    delete_list = []
    for page in pages:
        if 'Versions' in page:
            for version in page['Versions']:
                delete_list.append({
                    'Key': version['Key'],
                    'VersionId': version['VersionId']
                })
        
        if 'DeleteMarkers' in page:
            for marker in page['DeleteMarkers']:
                delete_list.append({
                    'Key': marker['Key'],
                    'VersionId': marker['VersionId']
                })
    
    if delete_list:
        # Delete object versions in batches of 1000
        for i in range(0, len(delete_list), 1000):
            batch = delete_list[i:i+1000]
            s3_client.delete_objects(
                Bucket=bucket_name,
                Delete={'Objects': batch}
            )
            print(f"Deleted {len(batch)} object versions from bucket")
    
    print(f"Successfully emptied bucket: {bucket_name}")

def delete_cloudformation_stacks():
    """Delete CloudFormation stacks."""
    cf_client = session.client('cloudformation')
    
    # Discover stacks
    response = cf_client.list_stacks(StackStatusFilter=['CREATE_COMPLETE', 'UPDATE_COMPLETE', 'ROLLBACK_COMPLETE'])
    stacks = [stack for stack in response['StackSummaries'] 
             if PROJECT_NAME.lower() in stack['StackName'].lower()]
    
    if not stacks:
        print("No CloudFormation stacks found")
        return
    
    print(f"Found {len(stacks)} CloudFormation stacks to delete")
    
    # Delete stacks
    for stack in stacks:
        stack_name = stack['StackName']
        try:
            print(f"Deleting stack: {stack_name}")
            cf_client.delete_stack(StackName=stack_name)
            print(f"Stack deletion initiated: {stack_name}")
            
            # Wait for stack deletion to complete
            print(f"Waiting for stack {stack_name} to be deleted...")
            waiter = cf_client.get_waiter('stack_delete_complete')
            waiter.wait(StackName=stack_name)
            print(f"Successfully deleted stack: {stack_name}")
        except Exception as e:
            print(f"Error deleting stack {stack_name}: {e}")

## Selective Cleanup

Let's create a user interface for selective cleanup of resources.

In [ ]:
# Create checkboxes for resource types
endpoints_checkbox = widgets.Checkbox(
    value=True,
    description='SageMaker Endpoints',
    style={'description_width': 'initial'}
)

endpoint_configs_checkbox = widgets.Checkbox(
    value=True,
    description='SageMaker Endpoint Configs',
    style={'description_width': 'initial'}
)

models_checkbox = widgets.Checkbox(
    value=True,
    description='SageMaker Models',
    style={'description_width': 'initial'}
)

pipelines_checkbox = widgets.Checkbox(
    value=True,
    description='SageMaker Pipelines',
    style={'description_width': 'initial'}
)

monitoring_checkbox = widgets.Checkbox(
    value=True,
    description='Monitoring Schedules',
    style={'description_width': 'initial'}
)

dashboards_checkbox = widgets.Checkbox(
    value=True,
    description='CloudWatch Dashboards',
    style={'description_width': 'initial'}
)

alarms_checkbox = widgets.Checkbox(
    value=True,
    description='CloudWatch Alarms',
    style={'description_width': 'initial'}
)

rules_checkbox = widgets.Checkbox(
    value=True,
    description='EventBridge Rules',
    style={'description_width': 'initial'}
)

s3_checkbox = widgets.Checkbox(
    value=False,
    description=f'Empty S3 Bucket ({DATA_BUCKET})',
    style={'description_width': 'initial'}
)

stacks_checkbox = widgets.Checkbox(
    value=False,
    description='CloudFormation Stacks',
    style={'description_width': 'initial'}
)

# Create select all checkbox
select_all_checkbox = widgets.Checkbox(
    value=False,
    description='Select All',
    style={'description_width': 'initial'}
)

# Function to handle select all
def on_select_all_change(change):
    if change['name'] == 'value':
        endpoints_checkbox.value = change['new']
        endpoint_configs_checkbox.value = change['new']
        models_checkbox.value = change['new']
        pipelines_checkbox.value = change['new']
        monitoring_checkbox.value = change['new']
        dashboards_checkbox.value = change['new']
        alarms_checkbox.value = change['new']
        rules_checkbox.value = change['new']
        s3_checkbox.value = change['new']
        stacks_checkbox.value = change['new']

select_all_checkbox.observe(on_select_all_change, names='value')

# Display checkboxes
display(select_all_checkbox)
display(widgets.HBox([endpoints_checkbox, endpoint_configs_checkbox, models_checkbox, pipelines_checkbox, monitoring_checkbox]))
display(widgets.HBox([dashboards_checkbox, alarms_checkbox, rules_checkbox, s3_checkbox, stacks_checkbox]))

In [ ]:
# Create function for selective cleanup
def selective_cleanup():
    print("Starting selective cleanup...\n")
    
    # Delete SageMaker endpoints
    if endpoints_checkbox.value:
        print("\n=== Deleting SageMaker Endpoints ===")
        delete_sagemaker_endpoints()
    
    # Delete SageMaker endpoint configs
    if endpoint_configs_checkbox.value:
        print("\n=== Deleting SageMaker Endpoint Configs ===")
        delete_sagemaker_endpoint_configs()
    
    # Delete SageMaker models
    if models_checkbox.value:
        print("\n=== Deleting SageMaker Models ===")
        delete_sagemaker_models()
    
    # Delete SageMaker pipelines
    if pipelines_checkbox.value:
        print("\n=== Deleting SageMaker Pipelines ===")
        delete_sagemaker_pipelines()
    
    # Delete monitoring schedules
    if monitoring_checkbox.value:
        print("\n=== Deleting Monitoring Schedules ===")
        delete_monitoring_schedules()
    
    # Delete CloudWatch dashboards
    if dashboards_checkbox.value:
        print("\n=== Deleting CloudWatch Dashboards ===")
        delete_cloudwatch_dashboards()
    
    # Delete CloudWatch alarms
    if alarms_checkbox.value:
        print("\n=== Deleting CloudWatch Alarms ===")
        delete_cloudwatch_alarms()
    
    # Delete EventBridge rules
    if rules_checkbox.value:
        print("\n=== Deleting EventBridge Rules ===")
        delete_eventbridge_rules()
    
    # Empty S3 bucket
    if s3_checkbox.value:
        print(f"\n=== Emptying S3 Bucket ({DATA_BUCKET}) ===")
        empty_s3_bucket(DATA_BUCKET)
    
    # Delete CloudFormation stacks
    if stacks_checkbox.value:
        print("\n=== Deleting CloudFormation Stacks ===")
        delete_cloudformation_stacks()
    
    print("\nSelective cleanup complete!")

# Create button for selective cleanup
selective_cleanup_button = widgets.Button(
    description='Cleanup Selected Resources',
    button_style='danger',
    tooltip='Click to clean up selected resources'
)

selective_cleanup_output = widgets.Output()

def on_selective_cleanup_clicked(b):
    with selective_cleanup_output:
        selective_cleanup_output.clear_output()
        selective_cleanup()

selective_cleanup_button.on_click(on_selective_cleanup_clicked)

display(selective_cleanup_button, selective_cleanup_output)

## Complete Cleanup

Let's create a function for complete cleanup of all resources.

In [ ]:
# Create function for complete cleanup
def complete_cleanup():
    print("Starting complete cleanup...\n")
    
    # Delete SageMaker endpoints
    print("\n=== Deleting SageMaker Endpoints ===")
    delete_sagemaker_endpoints()
    
    # Delete SageMaker endpoint configs
    print("\n=== Deleting SageMaker Endpoint Configs ===")
    delete_sagemaker_endpoint_configs()
    
    # Delete SageMaker models
    print("\n=== Deleting SageMaker Models ===")
    delete_sagemaker_models()
    
    # Delete SageMaker pipelines
    print("\n=== Deleting SageMaker Pipelines ===")
    delete_sagemaker_pipelines()
    
    # Delete monitoring schedules
    print("\n=== Deleting Monitoring Schedules ===")
    delete_monitoring_schedules()
    
    # Delete CloudWatch dashboards
    print("\n=== Deleting CloudWatch Dashboards ===")
    delete_cloudwatch_dashboards()
    
    # Delete CloudWatch alarms
    print("\n=== Deleting CloudWatch Alarms ===")
    delete_cloudwatch_alarms()
    
    # Delete EventBridge rules
    print("\n=== Deleting EventBridge Rules ===")
    delete_eventbridge_rules()
    
    # Empty S3 bucket
    print(f"\n=== Emptying S3 Bucket ({DATA_BUCKET}) ===")
    empty_s3_bucket(DATA_BUCKET)
    
    # Delete CloudFormation stacks
    print("\n=== Deleting CloudFormation Stacks ===")
    delete_cloudformation_stacks()
    
    print("\nComplete cleanup finished!")

# Create confirmation dialog for complete cleanup
def confirm_complete_cleanup():
    confirmation = widgets.Text(
        value='',
        placeholder='Type "confirm" to proceed',
        description='Confirmation:',
        style={'description_width': 'initial'}
    )
    
    confirm_button = widgets.Button(
        description='Confirm Complete Cleanup',
        button_style='danger',
        tooltip='Click to confirm complete cleanup'
    )
    
    confirm_output = widgets.Output()
    
    def on_confirm_clicked(b):
        with confirm_output:
            confirm_output.clear_output()
            if confirmation.value.lower() == 'confirm':
                complete_cleanup()
            else:
                print("Confirmation failed. Please type 'confirm' to proceed.")
    
    confirm_button.on_click(on_confirm_clicked)
    
    display(widgets.HTML("<b style='color:red'>WARNING: This will delete ALL resources created by the MLOps SageMaker Demo. This action cannot be undone.</b>"))
    display(confirmation, confirm_button, confirm_output)

# Create button to show confirmation dialog
complete_cleanup_button = widgets.Button(
    description='Complete Cleanup (All Resources)',
    button_style='danger',
    tooltip='Click to start complete cleanup process'
)

complete_cleanup_output = widgets.Output()

def on_complete_cleanup_clicked(b):
    with complete_cleanup_output:
        complete_cleanup_output.clear_output()
        confirm_complete_cleanup()

complete_cleanup_button.on_click(on_complete_cleanup_clicked)

display(complete_cleanup_button, complete_cleanup_output)

## Resource Verification

After cleanup, let's verify that all resources have been properly deleted.

In [ ]:
# Create function to verify cleanup
def verify_cleanup():
    print("Verifying cleanup...\n")
    
    resources = discover_resources()
    
    # Check if any resources remain
    remaining_resources = False
    for resource_type, items in resources.items():
        if items:
            remaining_resources = True
            print(f"\n{resource_type.replace('_', ' ').title()} ({len(items)}):\n")
            df = pd.DataFrame(items)
            display(df)
    
    if not remaining_resources:
        print("All resources have been successfully cleaned up!")
    else:
        print("\nSome resources still remain. You may need to delete them manually.")

# Create button to verify cleanup
verify_button = widgets.Button(
    description='Verify Cleanup',
    button_style='info',
    tooltip='Click to verify cleanup'
)

verify_output = widgets.Output()

def on_verify_clicked(b):
    with verify_output:
        verify_output.clear_output()
        verify_cleanup()

verify_button.on_click(on_verify_clicked)

display(verify_button, verify_output)

## Manual Cleanup Instructions

If automatic cleanup fails for any reason, here are manual instructions for cleaning up resources through the AWS Management Console.

### SageMaker Resources

1. **SageMaker Endpoints**
   - Go to the [SageMaker console](https://console.aws.amazon.com/sagemaker/)
   - Click on "Inference" in the left navigation
   - Click on "Endpoints"
   - Select endpoints with names containing "mlops-sagemaker-demo"
   - Click "Actions" > "Delete"

2. **SageMaker Models**
   - Go to the [SageMaker console](https://console.aws.amazon.com/sagemaker/)
   - Click on "Inference" in the left navigation
   - Click on "Models"
   - Select models with names containing "mlops-sagemaker-demo"
   - Click "Actions" > "Delete"

3. **SageMaker Pipelines**
   - Go to the [SageMaker console](https://console.aws.amazon.com/sagemaker/)
   - Click on "Pipelines" in the left navigation
   - Select pipelines with names containing "mlops-sagemaker-demo"
   - Click "Actions" > "Delete"

4. **SageMaker Monitoring Schedules**
   - Go to the [SageMaker console](https://console.aws.amazon.com/sagemaker/)
   - Click on "Inference" in the left navigation
   - Click on "Model Monitor"
   - Select monitoring schedules with names containing "mlops-sagemaker-demo"
   - Click "Actions" > "Delete"

### CloudWatch Resources

1. **CloudWatch Dashboards**
   - Go to the [CloudWatch console](https://console.aws.amazon.com/cloudwatch/)
   - Click on "Dashboards" in the left navigation
   - Select dashboards with names containing "mlops-sagemaker-demo"
   - Click "Actions" > "Delete"

2. **CloudWatch Alarms**
   - Go to the [CloudWatch console](https://console.aws.amazon.com/cloudwatch/)
   - Click on "Alarms" in the left navigation
   - Select alarms with names containing "mlops-sagemaker-demo"
   - Click "Actions" > "Delete"

### EventBridge Resources

1. **EventBridge Rules**
   - Go to the [EventBridge console](https://console.aws.amazon.com/events/)
   - Click on "Rules" in the left navigation
   - Select rules with names containing "mlops-sagemaker-demo"
   - Click "Delete"

### S3 Resources

1. **S3 Bucket**
   - Go to the [S3 console](https://console.aws.amazon.com/s3/)
   - Click on the bucket "lucaskle-ab3-project-pv"
   - Select all objects and click "Delete"
   - If versioning is enabled, click on "Show versions" and delete all versions

### CloudFormation Resources

1. **CloudFormation Stacks**
   - Go to the [CloudFormation console](https://console.aws.amazon.com/cloudformation/)
   - Select stacks with names containing "mlops-sagemaker-demo"
   - Click "Delete"

### Final Verification

After manual cleanup, verify that all resources have been deleted by checking each service console.

## Cost Optimization Checklist

Here's a checklist to ensure you're not incurring unnecessary costs:

In [ ]:
# Create cost optimization checklist
checklist_items = [
    {'category': 'SageMaker', 'item': 'Delete all SageMaker endpoints', 'critical': True},
    {'category': 'SageMaker', 'item': 'Delete all SageMaker notebook instances', 'critical': True},
    {'category': 'SageMaker', 'item': 'Delete all SageMaker models', 'critical': False},
    {'category': 'SageMaker', 'item': 'Delete all SageMaker pipelines', 'critical': False},
    {'category': 'SageMaker', 'item': 'Delete all monitoring schedules', 'critical': True},
    {'category': 'CloudWatch', 'item': 'Delete all CloudWatch dashboards', 'critical': False},
    {'category': 'CloudWatch', 'item': 'Delete all CloudWatch alarms', 'critical': False},
    {'category': 'EventBridge', 'item': 'Delete all EventBridge rules', 'critical': False},
    {'category': 'S3', 'item': 'Empty S3 bucket to avoid storage costs', 'critical': True},
    {'category': 'CloudFormation', 'item': 'Delete all CloudFormation stacks', 'critical': False},
    {'category': 'IAM', 'item': 'Verify IAM roles and policies are cleaned up', 'critical': False},
    {'category': 'Cost Explorer', 'item': 'Check Cost Explorer for any remaining costs', 'critical': True}
]

# Create DataFrame for checklist
checklist_df = pd.DataFrame(checklist_items)
checklist_df['Priority'] = checklist_df['critical'].map({True: 'High', False: 'Medium'})
checklist_df = checklist_df[['category', 'item', 'Priority']]
checklist_df.columns = ['Category', 'Item', 'Priority']

# Display checklist
display(HTML("<h3>Cost Optimization Checklist</h3>"))
display(checklist_df.style.apply(lambda x: ['background-color: #ffcccc' if x.Priority == 'High' else '' for i in x], axis=1))

## Conclusion

This notebook has provided a comprehensive cleanup procedure for all AWS resources created by the MLOps SageMaker Demo. By following these steps, you can ensure that you're not incurring unnecessary costs for resources that are no longer needed.

Remember to verify that all resources have been properly deleted after cleanup, and check Cost Explorer to ensure that there are no unexpected costs.